In [ ]:
import numpy as np
from sklearn.svm import SVC
import joblib


## Global Parameters



In [ ]:
KERNEL = "rbf"
GLOBAL_C = 1.0
GAMMA = "scale"
THRESHOLD = 0.6

## Load Features

In [ ]:
X = np.load("../features/X_hog_scaled.npy")
y = np.load("../features/y.npy")
print(X.shape, y.shape)

## Model Training

In [ ]:
svm = SVC(
    kernel=KERNEL,
    C=GLOBAL_C,
    gamma=GAMMA,
    probability=True
)

svm.fit(X, y)

## Save Model

In [ ]:
joblib.dump(svm, "../models/svm_model.pkl")

## Model Prediction with Rejection

In [ ]:
def svm_predict_with_rejection(model, X, threshold=0.6):
    probs = model.predict_proba(X)
    max_probs = np.max(probs, axis=1)
    preds = model.predict(X)

    preds[max_probs < threshold] = 6
    return preds
